In [1]:
# setup:
# 1. open scds/vst_tracker.scd file in SuperCollider
# and run first cell (on MacOS press 'Cmd+Enter')

# 2. run this cell
from setup import D

synth_count = 8
d = D(
    synth_count=synth_count,
    context='opl',
    mode='once',
)

In [2]:
from enum import Enum
import time

from domblar.sc3.client import SC3Client

class Event():
    def __init__(self, note, scale, timing, dur, synth_idx):
        self.note = note
        self.scale = scale
        self.timing = timing
        self.dur = dur
        self.synth_idx = synth_idx

class NoteChange():
    def __init__(self, timing, synth_idx, lpf, lpr):
        self.timing = timing
        self.synth_idx = synth_idx
        self.lpf = lpf
        self.lpr = lpr

class Notes(str, Enum):
    REST = '.'

class Events():
    def __init__(self, stream, scale):
        # NOTE:
        # - scale is global
        # - notation for duration is not finalized
        # - sustain is same as duration
        # - bpm is global
        # - no info about synth indices
        self.events = []
        cur_beat = 0
        for notes in stream:
            # convert to list
            chord = []
            dur = 1
            if isinstance(notes, str) and (notes != Notes.REST):
                notes, dur = notes.split(':')
                dur = int(dur)
                notes = eval(notes)
            match notes:
                case int():
                    chord = [notes]
                case Notes.REST:
                    chord = [Notes.REST]
                case tuple():
                    chord = list(notes)
                case _:
                    print(f'Wrong type for notes {notes}: {type(notes)}')
                    assert False

            for note_idx, note in enumerate(chord):
                self.events.append(Event(note, scale, cur_beat, dur,
                                         synth_idx=note_idx))
                sub_beat_count = 16
                for sub_beat in range(sub_beat_count):
                    self.events.append(NoteChange(cur_beat + sub_beat / sub_beat_count,
                                                  synth_idx=note_idx,
                                                  lpf=(1 - sub_beat / sub_beat_count) * 4000,
                                                  lpr=1 - sub_beat / sub_beat_count,))
            cur_beat += dur


def play(events: Events, client: SC3Client,
         dur=0.25,
         sus=None,
         voice_amps=[],
         ):

    cur_beat = 0
    for event in events.events:
        assert event.synth_idx < synth_count

        # NOTE: assuming events are sorted by timing
        assert cur_beat <= event.timing
        if event.timing > cur_beat:
            sleep_dur = dur * (event.timing - cur_beat)
            time.sleep(sleep_dur)
            cur_beat = event.timing

        timetag = time.time()

        if isinstance(event, NoteChange):
            timetag = time.time()
            client.modify_note(
                event.synth_idx,
                timetag=timetag,
                channel=0,
                lpf=event.lpf,
                lpr=event.lpr
                )
        else:
            send_note_dur = dur
            if sus:
                send_note_dur = sus
            send_note_dur *= event.dur

            amp = 1.0
            if voice_amps:
                amp *= voice_amps[event.synth_idx]  # TODO: should be voice_idx

            freq = event.scale.get_freq(event.note)
            client.send_note(
                # FIXME: synth_idx[note_idx] + last_reps[note_idx],
                event.synth_idx,
                freq=freq,
                dur=send_note_dur,
                amp=amp,
                timetag=timetag,
                channel=0,  # TODO: for MPE with channels use channel=note_idx
                )
    time.sleep(dur)


In [14]:
# acid
from domblar.ed import ED

def set_synths():
    for i in range(synth_count):
        d.set_synth(i, 'jpiano')
set_synths()

edo = 12
scale = ED(edo)

notes = [
    2-edo*2,
    # 2+7-edo*2,
]

print(notes)

for _ in range(1):
    play(Events(notes, scale=scale),
        d.client,
        # dur=1.,
        dur=.26,
        # dur=.33,
        # dur=.25,
        #  synth_idx=[0],
        )

[-22, -15, -22, -15, -22, -15, -22, -15]


In [15]:
# cleanup
d.stop_server()